In [1]:
import os
import pandas as pd
import glob
import nbformat

In [2]:
directory = 'C:/Users/FANNY MARIA D SOUZA/Capstone_Movement/Movement_CompletedSteps_SimplePie/'
file_pattern = directory + '*_Movement.ipynb'

#### Extracting Position Features

In [3]:
def extract_features_summary(file_path):
    with open(file_path, 'r') as file:
        nb = nbformat.read(file, as_version=4)
    
    local_context = {}
    for cell in nb.cells:
        if cell.cell_type == 'code':
            try:
                # Check if the cell contains plotting code and skip it
                if 'plt.show()' in cell.source or 'plt.plot' in cell.source or 'plt.bar' in cell.source:
                    continue
                exec(cell.source, {}, local_context)
                if 'total_distances_df' in local_context:
                    return local_context['total_distances_df']
            except Exception as e:
                print(f"Error executing cell in {file_path}: {e}")
    return None

total_distance = []

# Extract data from each notebook
for file_path in glob.glob(file_pattern):
    participant_id = os.path.basename(file_path)[:4] 
    df = extract_features_summary(file_path)
    if df is not None:
        df['Participant'] = participant_id 
        total_distance.append(df)
    else:
        print(f"No features_summary found in {file_path}")

combined_features_summary = pd.concat(total_distance, ignore_index=True)
combined_features_summary

,Tracker,Total Distance,Participant
0,Chest,3540.337685,C002
1,LeftShoulder,4748.732604,C002
2,RightShoulder,7406.876749,C002
3,LeftElbow,5769.977725,C002
4,RightElbow,9951.623287,C002
...,...,...,...
115,LeftElbow,3195.297103,C031
116,RightElbow,7318.365951,C031
117,MotionControllerLeft,3785.102563,C031
118,MotionControllerRight,10534.010814,C031


In [4]:
pivoted_df = combined_features_summary.pivot_table(
    index='Participant', 
    columns='Tracker', 
    values='Total Distance'
).reset_index()

pivoted_df


Tracker,Participant,Chest,HeadCamera,LeftElbow,LeftShoulder,MotionControllerLeft,MotionControllerRight,RightElbow,RightShoulder
0,C002,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749
1,C004,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120
2,C005,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569
3,C010,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912
4,C011,1016.264494,1066.769295,1683.153449,1419.085839,1624.778017,3841.361179,2158.849534,1827.696099
5,C014,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424
6,C015,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708
7,C018,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231
8,C019,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575
9,C022,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791


In [5]:
from sklearn.impute import KNNImputer
import numpy as np

imputer = KNNImputer(n_neighbors=1)

numerical_cols = pivoted_df.select_dtypes(include=['float64', 'int64']).columns
pivoted_df[numerical_cols] = pivoted_df[numerical_cols].replace(0, np.nan)

pivoted_df[numerical_cols] = imputer.fit_transform(pivoted_df[numerical_cols])
pivoted_df

Tracker,Participant,Chest,HeadCamera,LeftElbow,LeftShoulder,MotionControllerLeft,MotionControllerRight,RightElbow,RightShoulder
0,C002,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749
1,C004,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120
2,C005,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569
3,C010,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912
4,C011,1016.264494,1066.769295,1683.153449,1419.085839,1624.778017,3841.361179,2158.849534,1827.696099
5,C014,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424
6,C015,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708
7,C018,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231
8,C019,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575
9,C022,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791


In [6]:
new_column_names = {
    'Chest': 'Chest Total Distance',
    'HeadCamera': 'HeadCamera Total Distance',
    'LeftElbow': 'LeftElbow Total Distance',
    'LeftShoulder': 'LeftShoulder Total Distance',
    'MotionControllerLeft': 'MotionControllerLeft Total Distance',
    'MotionControllerRight': 'MotionControllerRight Total Distance',
    'RightElbow': 'RightElbow Total Distance',
    'RightShoulder': 'RightShoulder Total Distance'
}

# Rename columns
pivoted_df = pivoted_df.rename(columns=new_column_names)
pivoted_df

Tracker,Participant,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance
0,C002,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749
1,C004,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120
2,C005,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569
3,C010,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912
4,C011,1016.264494,1066.769295,1683.153449,1419.085839,1624.778017,3841.361179,2158.849534,1827.696099
5,C014,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424
6,C015,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708
7,C018,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231
8,C019,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575
9,C022,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791


In [7]:
pivoted_df.to_csv('PositionDistance_SimplePie.csv',index=False)

In [8]:
bscore=pd.read_csv('M_BDScore.csv')

In [9]:
final_df = pd.merge(bscore, pivoted_df, on='Participant', how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569
3,C010,54,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912
4,C011,34,1016.264494,1066.769295,1683.153449,1419.085839,1624.778017,3841.361179,2158.849534,1827.696099
5,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424
6,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708
7,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231
8,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575
9,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791


In [10]:
addapples=pd.read_csv('SimplePie_AddApples.csv')

In [11]:
if 'Block Design Raw Score' in addapples.columns:
    addapples['Block Design Raw Score'] = addapples['Block Design Raw Score'].astype(int)
addapples = addapples.drop(columns=['Step'])

final_df = pd.merge(final_df, addapples, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,Chest Distance - Add Apples,HeadCamera Distance - Add Apples,LeftElbow Distance - Add Apples,LeftShoulder Distance - Add Apples,MotionControllerLeft Distance - Add Apples,MotionControllerRight Distance - Add Apples,RightElbow Distance - Add Apples,RightShoulder Distance - Add Apples
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749,435.107928,624.805110,705.801971,563.893022,947.840128,2663.391550,1870.306193,1190.412091
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120,247.174945,293.060332,614.710946,477.332484,745.064010,854.962361,943.312282,585.492297
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569,511.557288,829.818348,1188.353996,910.710785,3034.224119,1608.006156,1072.593997,1183.886136
3,C010,54,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912,536.387704,627.323344,805.596982,722.040053,1033.639673,2413.612531,1515.126254,1013.578828
4,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424,739.680718,1079.493893,3588.904183,1338.030583,2871.806319,3881.368784,2451.963309,1636.366156
5,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708,846.967873,967.420050,1060.655016,969.086539,1284.899226,2994.697206,1927.376524,1370.348208
6,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231,475.244808,773.593897,999.382906,784.139287,1401.136589,2920.508881,1489.355663,945.365477
7,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575,207.500902,330.472059,405.405529,228.189871,1771.425292,110.935285,943.312282,562.875621
8,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791,975.477951,1390.003537,1377.548578,1309.612258,1469.001324,4089.335793,2405.021218,1837.846669
9,C023,52,3793.795816,4712.913044,5195.060891,4738.732267,5911.425372,11719.417729,8531.826845,5828.200870,824.231380,949.448955,1461.064887,1197.437699,1760.440472,2788.476796,1907.344113,1218.902954


In [12]:
addbutter=pd.read_csv('SimplePie_AddButter.csv')

In [13]:
if 'Block Design Raw Score' in addbutter.columns:
    addapples['Block Design Raw Score'] = addbutter['Block Design Raw Score'].astype(int)
addbutter = addbutter.drop(columns=['Step'])

final_df = pd.merge(final_df, addbutter, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Apples,RightShoulder Distance - Add Apples,Chest Distance - Add Butter,HeadCamera Distance - Add Butter,LeftElbow Distance - Add Butter,LeftShoulder Distance - Add Butter,MotionControllerLeft Distance - Add Butter,MotionControllerRight Distance - Add Butter,RightElbow Distance - Add Butter,RightShoulder Distance - Add Butter
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749,...,1870.306193,1190.412091,137.426355,194.928198,106.500109,127.648978,146.091674,589.989980,508.080445,347.615839
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120,...,943.312282,585.492297,131.913954,220.883048,297.924372,242.485275,389.657921,354.535972,118.391897,193.819130
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569,...,1072.593997,1183.886136,144.152977,201.578498,361.997470,271.998446,455.801352,425.674955,706.680071,236.430900
3,C010,54,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912,...,1515.126254,1013.578828,107.651726,138.543294,285.250598,209.287920,413.009446,134.522255,126.195299,112.447679
4,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424,...,2451.963309,1636.366156,465.928089,595.513026,512.944117,492.948570,626.155193,1372.302924,1020.634290,780.666999
5,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708,...,1927.376524,1370.348208,99.397880,116.957918,143.628001,123.477562,184.418438,369.580931,286.690272,186.378962
6,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231,...,1489.355663,945.365477,123.371122,221.941791,308.163571,236.663478,399.961616,299.740678,203.773024,169.807244
7,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575,...,943.312282,562.875621,99.397880,102.319814,143.628001,64.844725,358.636328,369.580931,241.756197,160.074396
8,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791,...,2405.021218,1837.846669,550.278136,778.272907,927.453746,802.883420,1046.135840,1775.603941,1238.104785,968.315960
9,C023,52,3793.795816,4712.913044,5195.060891,4738.732267,5911.425372,11719.417729,8531.826845,5828.200870,...,1907.344113,1218.902954,154.653961,219.788516,196.411781,185.870785,196.642141,640.698442,437.480695,158.337560


In [15]:
addcinnamon=pd.read_csv('SimplePie_AddCinnamon.csv')

In [16]:
if 'Block Design Raw Score' in addcinnamon.columns:
    addcinnamon['Block Design Raw Score'] = addcinnamon['Block Design Raw Score'].astype(int)
addcinnamon = addcinnamon.drop(columns=['Step'])

final_df = pd.merge(final_df, addcinnamon, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Butter,RightShoulder Distance - Add Butter,Chest Distance - Add Cinnamon,HeadCamera Distance - Add Cinnamon,LeftElbow Distance - Add Cinnamon,LeftShoulder Distance - Add Cinnamon,MotionControllerLeft Distance - Add Cinnamon,MotionControllerRight Distance - Add Cinnamon,RightElbow Distance - Add Cinnamon,RightShoulder Distance - Add Cinnamon
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749,...,508.080445,347.615839,176.543370,222.538848,255.328186,240.098530,281.452259,598.823083,455.591867,358.320554
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120,...,118.391897,193.819130,69.706134,157.902359,52.486875,68.742924,38.483365,367.670139,282.942319,194.082550
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569,...,706.680071,236.430900,393.566572,853.433980,730.035698,706.564382,3841.270968,1089.391433,707.497659,889.320691
3,C010,54,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912,...,126.195299,112.447679,348.746848,372.007400,501.848924,407.545462,664.185906,867.412548,639.626106,485.916721
4,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424,...,1020.634290,780.666999,112.372573,141.774113,227.713051,202.633559,369.344525,696.602321,556.001817,359.498240
5,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708,...,286.690272,186.378962,136.076275,172.340377,149.753145,146.064752,152.876623,362.359136,267.695037,194.382029
6,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231,...,203.773024,169.807244,64.326650,116.648961,69.380677,66.791835,89.819210,371.295480,282.942319,183.458120
7,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575,...,241.756197,160.074396,242.023597,314.452562,627.743619,422.422681,882.320816,1030.672886,567.649296,363.854230
8,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791,...,1238.104785,968.315960,130.481410,186.332860,178.961547,175.011948,155.481596,402.632871,281.499340,225.244273
9,C023,52,3793.795816,4712.913044,5195.060891,4738.732267,5911.425372,11719.417729,8531.826845,5828.200870,...,437.480695,158.337560,75.620822,100.327400,135.944699,111.642167,145.363306,210.459596,138.528195,113.452079


In [18]:
addcrusttop=pd.read_csv('SimplePie_AddCrustTop.csv')

In [19]:
if 'Block Design Raw Score' in addcrusttop.columns:
    addcrusttop['Block Design Raw Score'] = addcrusttop['Block Design Raw Score'].astype(int)
addcrusttop = addcrusttop.drop(columns=['Step'])

final_df = pd.merge(final_df, addcrusttop, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Cinnamon,RightShoulder Distance - Add Cinnamon,Chest Distance - Add Crusttop,HeadCamera Distance - Add Crusttop,LeftElbow Distance - Add Crusttop,LeftShoulder Distance - Add Crusttop,MotionControllerLeft Distance - Add Crusttop,MotionControllerRight Distance - Add Crusttop,RightElbow Distance - Add Crusttop,RightShoulder Distance - Add Crusttop
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749,...,455.591867,358.320554,360.315844,491.358404,774.073914,600.747154,961.973069,1211.884444,931.091523,717.321184
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120,...,282.942319,194.082550,139.675576,288.360246,394.479865,301.143706,538.717189,330.320840,224.956147,220.192450
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569,...,707.497659,889.320691,56.901104,110.299522,93.322110,91.630500,325.461920,69.909413,144.506222,195.593751
3,C010,54,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912,...,639.626106,485.916721,242.861087,273.040864,432.151793,353.209896,569.980302,715.391916,496.599415,345.312053
4,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424,...,556.001817,359.498240,129.400866,165.235262,552.197715,219.320445,368.823296,436.997454,347.407777,230.671859
5,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708,...,267.695037,194.382029,263.613792,359.860502,309.033983,311.921119,283.010013,617.506904,503.229457,406.718852
6,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231,...,282.942319,183.458120,77.729065,148.757117,113.207691,96.022745,101.715698,517.764006,370.341684,239.953540
7,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575,...,567.649296,363.854230,143.077504,197.756925,281.300912,226.541034,387.526666,881.089580,627.494689,368.058107
8,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791,...,281.499340,225.244273,369.265942,453.681424,535.320317,482.845447,490.933110,982.043881,747.805006,607.182973
9,C023,52,3793.795816,4712.913044,5195.060891,4738.732267,5911.425372,11719.417729,8531.826845,5828.200870,...,138.528195,113.452079,179.159644,225.684684,226.651162,222.848127,194.875458,518.581086,426.112731,321.399603


In [20]:
addsugar=pd.read_csv('SimplePie_AddSugar.csv')

In [21]:
if 'Block Design Raw Score' in addsugar.columns:
    addsugar['Block Design Raw Score'] = addsugar['Block Design Raw Score'].astype(int)
addsugar = addsugar.drop(columns=['Step'])

final_df = pd.merge(final_df, addsugar, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Crusttop,RightShoulder Distance - Add Crusttop,Chest Distance - Add Sugar,HeadCamera Distance - Add Sugar,LeftElbow Distance - Add Sugar,LeftShoulder Distance - Add Sugar,MotionControllerLeft Distance - Add Sugar,MotionControllerRight Distance - Add Sugar,RightElbow Distance - Add Sugar,RightShoulder Distance - Add Sugar
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749,...,931.091523,717.321184,1219.616559,1523.925405,2250.325627,1662.424118,2844.636220,3298.153806,2615.483562,1900.866426
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120,...,224.956147,220.192450,834.634076,1270.300473,1794.529046,1250.381908,2174.306553,1572.653802,1121.966885,875.340386
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569,...,144.506222,195.593751,651.764749,992.870694,912.434261,895.313499,1986.045627,1096.984592,1797.140450,1300.105150
3,C010,54,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912,...,496.599415,345.312053,891.446470,1260.546475,1441.743448,1127.100325,1798.104683,3013.718244,2055.521326,1381.385164
4,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424,...,347.407777,230.671859,524.253096,546.827888,981.901698,753.859723,1504.195730,1212.990609,892.038747,698.705910
5,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708,...,503.229457,406.718852,393.568617,328.129303,493.936647,460.010155,548.517445,1022.619151,818.355667,613.929269
6,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231,...,370.341684,239.953540,1287.271687,1688.517361,2135.886985,1743.068755,2505.423895,2433.836189,1523.827369,1330.136232
7,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575,...,627.494689,368.058107,180.132028,428.556410,695.626225,351.368776,808.390077,131.023879,837.637312,547.770586
8,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791,...,747.805006,607.182973,1309.780075,1482.960198,1250.464466,1256.406369,1446.831681,3457.792216,2469.648496,1874.958341
9,C023,52,3793.795816,4712.913044,5195.060891,4738.732267,5911.425372,11719.417729,8531.826845,5828.200870,...,426.112731,321.399603,519.632316,525.054737,734.618234,692.618955,717.350571,1437.952602,1258.753167,912.422757


In [22]:
addwater=pd.read_csv('SimplePie_AddWater.csv')

In [23]:
if 'Block Design Raw Score' in addwater.columns:
    addwater['Block Design Raw Score'] = addwater['Block Design Raw Score'].astype(int)
addwater = addwater.drop(columns=['Step'])

final_df = pd.merge(final_df, addwater, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Sugar,RightShoulder Distance - Add Sugar,Chest Distance - Add Water,HeadCamera Distance - Add Water,LeftElbow Distance - Add Water,LeftShoulder Distance - Add Water,MotionControllerLeft Distance - Add Water,MotionControllerRight Distance - Add Water,RightElbow Distance - Add Water,RightShoulder Distance - Add Water
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749,...,2615.483562,1900.866426,329.647148,502.921514,505.883802,423.661252,728.400257,1309.677817,1045.173195,704.894875
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120,...,1121.966885,875.340386,117.885608,100.990335,38.264498,44.909267,51.333230,247.049190,240.483979,132.865290
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569,...,1797.140450,1300.105150,83.657887,141.528330,137.890749,109.749575,297.866511,157.899588,123.943912,153.619908
3,C010,54,3536.785029,4337.159624,5563.083264,4612.295155,7148.163559,10739.234159,7465.590741,5310.335912,...,2055.521326,1381.385164,73.323611,108.266514,137.539111,116.855355,151.253066,250.349107,163.639957,121.165573
4,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424,...,892.038747,698.705910,309.058277,402.615949,698.749709,513.578641,890.398933,462.222211,399.909934,363.136431
5,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708,...,818.355667,613.929269,178.869424,245.497832,220.360858,208.987170,238.392150,480.055713,402.974862,307.540889
6,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231,...,1523.827369,1330.136232,101.151571,158.094248,266.901700,187.361620,321.150599,238.574214,150.059087,124.157694
7,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575,...,837.637312,547.770586,73.323611,80.126575,137.539111,45.907536,223.479588,250.349107,167.816392,115.468651
8,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791,...,2469.648496,1874.958341,174.132811,253.887137,264.460663,244.667075,279.273170,484.040647,331.992316,248.263299
9,C023,52,3793.795816,4712.913044,5195.060891,4738.732267,5911.425372,11719.417729,8531.826845,5828.200870,...,1258.753167,912.422757,196.750273,236.297602,276.615357,241.404938,301.760801,410.348342,311.120305,236.695654


In [24]:
addpie=pd.read_csv('SimplePie_Putpieinoven.csv')

In [25]:
if 'Block Design Raw Score' in addpie.columns:
    addpie['Block Design Raw Score'] = addpie['Block Design Raw Score'].astype(int)
addpie = addpie.drop(columns=['Step'])

final_df = pd.merge(final_df, addpie, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Water,RightShoulder Distance - Add Water,Chest Distance - Add Pie,HeadCamera Distance - Add Pie,LeftElbow Distance - Add Pie,LeftShoulder Distance - Add Pie,MotionControllerLeft Distance - Add Pie,MotionControllerRight Distance - Add Pie,RightElbow Distance - Add Pie,RightShoulder Distance - Add Pie
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749,...,1045.173195,704.894875,396.839522,588.972212,633.813499,577.085352,694.780765,1206.768843,904.789629,804.099361
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120,...,240.483979,132.865290,541.747725,468.596219,1015.861546,571.060358,1043.588098,448.732209,306.457588,315.926396
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569,...,123.943912,153.619908,844.406083,1361.269495,1478.000290,1547.566629,2441.203333,1700.842512,1480.698647,981.417434
3,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424,...,399.909934,363.136431,352.406773,463.945048,1271.244249,447.498455,811.427516,959.576745,761.348568,590.221240
4,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708,...,402.974862,307.540889,556.256274,503.822225,613.869942,545.769429,681.398567,979.934415,889.555301,686.715558
5,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231,...,150.059087,124.157694,365.654213,518.550213,333.895844,468.289618,498.591734,896.275358,783.709057,628.644836
6,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575,...,167.816392,115.468651,185.570837,274.748966,132.523764,172.869095,171.422058,747.038614,592.252997,394.682842
7,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791,...,331.992316,248.263299,1342.898976,1598.243801,1771.352855,1531.615579,2156.372327,3308.074831,2179.900532,1826.747869
8,C023,52,3793.795816,4712.913044,5195.060891,4738.732267,5911.425372,11719.417729,8531.826845,5828.200870,...,311.120305,236.695654,1167.679512,1513.222793,1349.487421,1302.803182,1498.734292,2936.055188,2206.051478,1682.072292
9,C026,33,3712.412837,5749.058570,3580.337633,4319.972996,4137.744922,17425.234561,13313.769973,8477.196792,...,351.696977,222.010808,460.076559,696.923894,425.891651,504.448292,288.931966,1409.371767,1159.020858,878.905208


In [26]:
poursyrup=pd.read_csv('SimplePie_PourSyrup.csv')

In [27]:
if 'Block Design Raw Score' in poursyrup.columns:
    poursyrup['Block Design Raw Score'] = poursyrup['Block Design Raw Score'].astype(int)
poursyrup = poursyrup.drop(columns=['Step'])

final_df = pd.merge(final_df, poursyrup, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Pie,RightShoulder Distance - Add Pie,Chest Distance - Add Syrup,HeadCamera Distance - Add Syrup,LeftElbow Distance - Add Syrup,LeftShoulder Distance - Add Syrup,MotionControllerLeft Distance - Add Syrup,MotionControllerRight Distance - Add Syrup,RightElbow Distance - Add Syrup,RightShoulder Distance - Add Syrup
0,C002,28,3540.337685,4920.962251,5769.977725,4748.732604,7305.247009,12874.805034,9951.623287,7406.876749,...,904.789629,804.099361,107.383091,128.375084,97.672148,96.610705,134.679142,415.805498,345.384135,325.081398
1,C004,30,2577.946109,3582.167852,5175.122021,3794.190422,6173.798725,5728.355616,2380.548588,3401.852120,...,306.457588,315.926396,49.543192,83.195782,106.694101,85.978718,144.538303,156.798251,156.697908,150.365591
2,C005,46,3101.189815,5242.910113,5729.878784,5343.179186,14551.018086,7150.648100,7233.666213,5496.841569,...,1480.698647,981.417434,57.859381,99.620427,208.978326,127.631988,228.119027,236.999938,137.001598,137.781302
3,C014,32,3352.784517,4785.264969,9807.955035,5351.594257,9710.043987,12345.539981,9105.250967,6666.490424,...,761.348568,590.221240,65.534560,107.768708,198.079943,104.763839,220.986000,204.747386,137.001598,102.685641
4,C015,33,3357.411882,3933.139281,4010.717193,3817.265229,4359.665984,9270.873866,7056.302227,5281.368708,...,889.555301,686.715558,193.263113,260.838161,279.699088,272.987460,227.375039,424.114480,438.506891,379.405247
5,C018,50,3157.765553,4684.105489,5230.733667,4447.791272,6727.656318,10388.141859,6715.892551,4911.885231,...,783.709057,628.644836,70.586055,160.813435,155.813863,128.426135,178.764766,312.443413,208.918910,135.363181
6,C019,40,1383.847297,2338.284810,2423.513426,1922.001229,4866.029531,4861.301072,5490.140021,3478.463575,...,592.252997,394.682842,57.807381,76.317682,34.719657,47.969389,36.482578,333.557224,240.517588,157.968698
7,C022,32,5723.482644,7363.255247,7339.619020,6814.769553,8178.021958,17749.288619,11668.909577,9151.521791,...,2179.900532,1826.747869,141.039884,210.598673,193.390328,186.396613,198.685537,423.952220,328.404612,275.446742
8,C023,52,3793.795816,4712.913044,5195.060891,4738.732267,5911.425372,11719.417729,8531.826845,5828.200870,...,2206.051478,1682.072292,118.802037,165.675059,149.336368,133.426612,178.515678,598.065636,369.515065,267.508558
9,C026,33,3712.412837,5749.058570,3580.337633,4319.972996,4137.744922,17425.234561,13313.769973,8477.196792,...,1159.020858,878.905208,112.336355,208.520726,142.598124,134.730377,134.689515,638.976999,500.764698,306.158422


In [28]:
final_df.to_csv('Features_SimplePie_Movement.csv',index=False)